In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import datetime as dt
from datetime import datetime


from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer

import scipy.stats as stats
from scipy.stats import chi2_contingency

/Users/joohlee/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 1. Import traininng data and test dataset 

In [2]:
datasets = pd.read_csv('/Users/joohlee/Desktop/Cousera/Data_Science_at_Scale/Material/datasci_course_materials/kaggle/train.csv',delimiter=',')
train_data, test_data = train_test_split(datasets, test_size=0.2)


In [3]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
237,238,1,2,"Collyer, Miss. Marjorie ""Lottie""",female,8,0,2,C.A. 31921,26.2500,NaN,S
387,388,1,2,"Buss, Miss. Kate",female,36,0,0,27849,13.0000,NaN,S
545,546,0,1,"Nicholson, Mr. Arthur Ernest",male,64,0,0,693,26.0000,NaN,S
659,660,0,1,"Newell, Mr. Arthur Webster",male,58,0,2,35273,113.2750,D48,C
200,201,0,3,"Vande Walle, Mr. Nestor Cyriel",male,28,0,0,345770,9.5000,NaN,S
460,461,1,1,"Anderson, Mr. Harry",male,48,0,0,19952,26.5500,E12,S
216,217,1,3,"Honkanen, Miss. Eliina",female,27,0,0,STON/O2. 3101283,7.9250,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.0000,B42,S
119,120,0,3,"Andersson, Miss. Ellis Anna Maria",female,2,4,2,347082,31.2750,NaN,S
429,430,1,3,"Pickard, Mr. Berk (Berk Trembisky)",male,32,0,0,SOTON/O.Q. 392078,8.0500,E10,S


# 2. Feature selection. Chi Square test to find important features(independent variables) from datasets

In [4]:
features = ['Pclass','Name','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked','Ticket']
for feature in features:
    X = train_data[feature].astype(str)
    Y = train_data['Survived'].astype(str)    
    dfObserved = pd.crosstab(Y,X) 
    chi2, p, dof, expected = stats.chi2_contingency(dfObserved.values)
    if p < 0.05:
        print feature, p

Pclass 1.58160936395e-18
Sex 6.96285196295e-47
SibSp 3.28249624506e-06
Parch 3.23793431902e-05
Fare 5.57773698206e-09
Cabin 1.20468262863e-05
Embarked 1.38792598263e-05
Ticket 0.0250629266475


# 3. Create train data and test data with features and response

In [5]:
x_train_data2 = train_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
x_train_data2 = x_train_data2.drop('Sex', 1).join(x_train_data2['Sex'].str.get_dummies())
x_train_data2 = x_train_data2.drop('Embarked', 1).join(x_train_data2['Embarked'].str.get_dummies())

y_train_data2 = train_data[['Survived']]
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(x_train_data2)
x_train_data2_imp = imp.transform(x_train_data2)

In [7]:
x_train_data2_imp

array([[  2.,   8.,   0., ...,   0.,   0.,   1.],
       [  2.,  36.,   0., ...,   0.,   0.,   1.],
       [  1.,  64.,   0., ...,   0.,   0.,   1.],
       ..., 
       [  3.,  26.,   0., ...,   0.,   0.,   1.],
       [  1.,  35.,   0., ...,   0.,   0.,   1.],
       [  3.,  26.,   1., ...,   0.,   0.,   1.]])

# 4. Random Forest modeling

In [8]:
randomF = RandomForestClassifier(n_estimators=100)
train_feature = x_train_data2_imp
train_target = y_train_data2

randomF = randomF.fit(train_feature, train_target)

/Users/joohlee/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [9]:
x_test_data2 = test_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
x_test_data2 = x_test_data2.drop('Sex', 1).join(x_test_data2['Sex'].str.get_dummies())
x_test_data2 = x_test_data2.drop('Embarked', 1).join(x_test_data2['Embarked'].str.get_dummies())
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(x_test_data2)
x_test_data2_imp = imp.transform(x_test_data2)

In [10]:
x_test_data2

,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
79,3,30.00,0,0,12.4750,1,0,0,0,1
817,2,31.00,1,1,37.0042,0,1,1,0,0
875,3,15.00,0,0,7.2250,1,0,1,0,0
411,3,NaN,0,0,6.8583,0,1,0,1,0
194,1,44.00,0,0,27.7208,1,0,1,0,0
0,3,22.00,1,0,7.2500,0,1,0,0,1
225,3,22.00,0,0,9.3500,0,1,0,0,1
34,1,28.00,1,0,82.1708,0,1,1,0,0
577,1,39.00,1,0,55.9000,1,0,0,0,1
28,3,NaN,0,0,7.8792,1,0,0,1,0


In [11]:
test_features = x_test_data2_imp
problem_pred_randomf = randomF.predict(test_features)

In [12]:
accuracy_randomforest = accuracy_score(test_data['Survived'],problem_pred_randomf)

In [13]:
accuracy_randomforest

0.77094972067039103

# 5. SVM modeling

In [14]:
C= 1.0
svc = svm.SVC(kernel='linear', C=C).fit(train_feature, train_target)

/Users/joohlee/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
problem_pred_svm = svc.predict(test_features)

In [16]:
accuracy_svm = accuracy_score(test_data['Survived'],problem_pred_svm)
accuracy_svm

0.78212290502793291

# 6. Tuning the parameters for SVM and Random Forest model 

In [19]:
C = [0.1,1.0,10.0,100.0]
for c in C:
    svc = svm.SVC(kernel='linear', C=c).fit(train_feature, train_target)
    problem_pred_svm = svc.predict(test_features)
    accuracy_svm = accuracy_score(test_data['Survived'],problem_pred_svm)
    print c, accuracy_svm


0.1 0.782122905028
1.0 0.782122905028
10.0 0.782122905028
100.0 0.776536312849


In [18]:
estimators = [10,50,100,500,1000]
for estimator in estimators:
    randomF = RandomForestClassifier(n_estimators=estimator)
    randomF = randomF.fit(train_feature, train_target)
    problem_pred_randomf = randomF.predict(test_features)
    accuracy_randomforest = accuracy_score(test_data['Survived'],problem_pred_randomf)
    print estimator, accuracy_randomforest

10 0.743016759777
50 0.748603351955
100 0.743016759777
500 0.759776536313
1000 0.759776536313


/Users/joohlee/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


# 7. Feature Engineering

In [38]:
datasets

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ticket_isalphabetic
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,True
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,False
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,False
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,False
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S,False
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.0750,NaN,S,False
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,NaN,S,False
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,NaN,C,False


In [30]:
datasets['ticket_isalphabetic']= datasets['Ticket'].str.contains('[a-zA-Z]')

In [32]:
datasets
train_data, test_data = train_test_split(datasets, test_size=0.2)

In [33]:
X = train_data['ticket_isalphabetic'].astype(str)
Y = train_data['Survived'].astype(str)    
dfObserved = pd.crosstab(Y,X) 
chi2, p, dof, expected = stats.chi2_contingency(dfObserved.values)
print feature, p

ticket_isalphabetic 0.78825654783


# 8. Create new Modeling with new features.

In [34]:
x_train_data2 = train_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','ticket_isalphabetic']]
x_train_data2 = x_train_data2.drop('Sex', 1).join(x_train_data2['Sex'].str.get_dummies())
x_train_data2 = x_train_data2.drop('Embarked', 1).join(x_train_data2['Embarked'].str.get_dummies())

y_train_data2 = train_data[['Survived']]
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(x_train_data2)
x_train_data2_imp = imp.transform(x_train_data2)

In [35]:
train_feature = x_train_data2_imp
train_target = y_train_data2
C= 1.0
svc = svm.SVC(kernel='linear', C=C).fit(train_feature, train_target)

In [37]:
x_test_data2 = test_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','ticket_isalphabetic']]
x_test_data2 = x_test_data2.drop('Sex', 1).join(x_test_data2['Sex'].str.get_dummies())
x_test_data2 = x_test_data2.drop('Embarked', 1).join(x_test_data2['Embarked'].str.get_dummies())
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(x_test_data2)
x_test_data2_imp = imp.transform(x_test_data2)
test_features = x_test_data2_imp
problem_pred_svm = svc.predict(test_features)
accuracy_svm = accuracy_score(test_data['Survived'],problem_pred_svm)
accuracy_svm


0.8044692737430168